---
title: "Plotting made easy with hvPlot: 0.11 release"
date: "2024-09-27"
description: "Release announcement for hvPlot 0.11, including: DuckDB integration, automatic lat/lon conversion on tiled maps, subcoordinate-y axis support, and more!"
author: "Maxime Liquet"
categories: [release, hvplot]
image: "images/thumbnail.png"
---

<p style="text-align:center;"><img src="./images/diagram.svg" width="60%"></img></p>

## What is hvPlot?

hvPlot is an open-source library that offers powerful high-level functionality for data exploration and visualization that doesn't require you to learn a new API. You can get powerful interactive and compositional [Bokeh](https://bokeh.org), [Matplotlib](https://matplotlib.org), or [Plotly](https://plotly.com/python) plots by simply replacing `.plot` with `.hvplot`. hvPlot makes all the analytical power of the [HoloViz](https://holoviz.org) ecosystem available, using the APIs you already know.

## New release!

We are very pleased to announce the 0.11 release of [hvPlot](https://hvplot.holoviz.org)! We'll describe the main changes, including:

- New integration: DuckDB!
- Automatic latitude/longitude conversion when displaying a tile map
- Support for displaying subcoordinate y-axis
- New hover options: `hover_tooltips` and `hover_formatters`
- Optimized Pandas index support
- Fixing "No output in jupyter"
- Update of the minimum version of the dependencies

As usual the full change log is available on [GitHub](https://github.com/holoviz/hvplot/releases/tag/v0.11.0).

Many thanks to @Azaya89, @liufeimath and @philipc2 for their first contributions, to @iuryt for contributing again, and to the maintainers @ahuang11, @hoxbro, @maximlt and @philippjfr!

<hr>

You can install hvPlot with `pip install hvplot`, or with `conda install hvplot` (or `conda install conda-forge::hvplot`) if you are using [Anaconda](https://www.anaconda.com/download).

<hr>

🌟 An easy way to support hvPlot is to give it a [star on Github](https://github.com/holoviz/hvplot)! 🌟

## New integration: DuckDB!

hvPlot has added [DuckDB](https://duckdb.org/) to the [long list of libraries](https://hvplot.holoviz.org/user_guide/Integrations.html) it integrates with. Thanks [Andrew](https://github.com/ahuang11)!

Install DuckDB with `pip install duckdb` or `conda install conda-forge::python-duckdb` and import `hvplot.duckdb` to enable the integration. `.hvplot()` supports [DuckDB](https://duckdb.org/docs/api/python/overview.html) `DuckDBPyRelation` and `DuckDBConnection` objects. In the example below, we create a DuckDB in-memory connection (from a Pandas DataFrame to make it simple) and just plot it with `.hvplot.line(...)`.

In [2]:
import duckdb
import numpy as np
import pandas as pd
import hvplot.duckdb  # noqa 

df_pandas = pd.DataFrame(np.random.randn(1000, 4), columns=list('ABCD')).cumsum()
connection = duckdb.connect(':memory:')
relation = duckdb.from_df(df_pandas, connection=connection)
relation.to_view("example_view");
relation.describe()

┌─────────┬─────────────────────┬────────────────────┬─────────────────────┬─────────────────────┐
│  aggr   │          A          │         B          │          C          │          D          │
│ varchar │       double        │       double       │       double        │       double        │
├─────────┼─────────────────────┼────────────────────┼─────────────────────┼─────────────────────┤
│ count   │              1000.0 │             1000.0 │              1000.0 │              1000.0 │
│ mean    │   7.930371322885783 │  36.35360951552763 │  24.154735908915978 │  -13.00362507826071 │
│ stddev  │  16.008461148609722 │ 27.960019095827207 │  10.299388417832068 │   5.923391409471744 │
│ min     │ -21.305391040769113 │ -8.453728666960362 │ -1.3631250672343924 │ -29.076492513752164 │
│ max     │   35.74439674552896 │  90.80818555015034 │  42.752087388478024 │   3.115467700708284 │
│ median  │   4.370362016845963 │  43.39469899226518 │   24.53286588434336 │  -13.67168553725746 │
└─────────

In [3]:
relation.hvplot.line(y=['A', 'B', 'C', 'D'])

:NdOverlay   [Variable]
   :Curve   [index]   (value)

`DuckDBPyRelation` is a bit more optimized because it handles column subsetting directly within DuckDB before the data is converted to a `pd.DataFrame`. So, it's a good idea to use the `connection.sql()` method when possible, which gives you a `DuckDBPyRelation`, instead of `connection.execute()`, which returns a `DuckDBPyConnection`.

In [4]:
sql_expr = "SELECT * FROM example_view WHERE A > 0 AND B > 0"
connection.sql(sql_expr).hvplot.line(y=['A', 'B'], hover_cols=["C"])  # subsets A, B, C

:NdOverlay   [Variable]
   :Curve   [index]   (value,C)

## Automatic latitude/longitude conversion when displaying a tile map

A pretty common situation when dealing with geographic data is to have the data expressed in terms of latitude/longitude (e.g. `(52.520008°, 13.404954°)` for Berlin), typically GPS coordinates. To display this data on a tile map (think Google Map), it needs to be projected to the Pseudo-Mercator projection that is the de facto standard for Web mapping applications (e.g. `(6894701.26m, 1492232.65m)`for Berlin). Up until this release, you could perform that projection by:

- installing [GeoViews](https://geoviews.org/) and setting `geo=True`, or
- projecting the data yourself with a utility available in [HoloViews](https://holoviews.org/) (`from holoviews.util.transform import lon_lat_to_easting_northing`)

With this release and when you set `tiles`, hvPlot projects latitude/longitude ([EPSG:4326](https://epsg.io/4326) / WGS84) to easting/northing ([EPSG:3857](https://epsg.io/3857) / Pseudo-Mercator) coordinates without additional package dependencies if it detects that the values falls within expected latitude/longitude ranges. This automatic projection can be disabled with `projection=False`. Find out more in the [Geographic Guide](https://hvplot.holoviz.org/user_guide/Geographic_Data.html#tiled-web-map).

In [5]:
import hvplot.pandas  # noqa
from bokeh.sampledata.airport_routes import airports

airports.head(2)

,AirportID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ,Type,source
0,3411,Barter Island LRRS Airport,Barter Island,United States,BTI,PABA,70.134003,-143.582001,2,-9,A,America/Anchorage,airport,OurAirports
1,3413,Cape Lisburne LRRS Airport,Cape Lisburne,United States,LUR,PALU,68.875099,-166.110001,16,-9,A,America/Anchorage,airport,OurAirports


In [6]:
airports.hvplot.points('Longitude', 'Latitude', tiles=True, color='red', alpha=0.2)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]

## Support for displaying subcoordinate y-axis

hvPlot enables you to create overlays where each element has its own distinct y-axis subcoordinate system (added in HoloViews 0.18.0). To activate this feature that automatically distributes overlay elements along the y-axis, set the `subcoordinate_y` keyword to `True`. For example, this feature is particularly useful to analyse multiple timeseries.

In [7]:
import numpy as np
import hvplot.pandas  # noqa
from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature as sst

sst = sst.assign(locations=np.random.choice(['loc1', 'loc2', 'loc3', 'loc4'], size=len(sst)))
sst.head(2)

,temperature,locations
time,,
2016-02-15 00:00:00+00:00,4.929,loc2
2016-02-15 00:30:00+00:00,4.887,loc1


In [10]:
sst.hvplot(by='locations', subcoordinate_y=True)

:NdOverlay   [locations]
   :Curve   [time]   (temperature)

Try zooming in the plot above, the y-axis wheel-zoom will apply to each curve's respective sub-coordinate y-axis, rather than the global coordinate frame.

`subcoordinate_y` also accepts a dictionary of related options, for example set `subcoordinate_y={'subcoordinate_scale': 2}` to increase the scale of each sub-plot, resulting in each curve's vertical range overlapping 50% with its adjacent elements, which allows creating simple [ridge plots](https://en.wikipedia.org/wiki/Ridgeline_plot). Let us know in this [Github issue](https://github.com/holoviz/hvplot/issues/1342) if you'd be interested in a more extensive API to generate ridge plots.

In [11]:
df = pd.DataFrame({'value': np.random.randn(200), 'cat': list("ABCD") * 50})
df['value'] += df['cat'].map(ord)
df.hvplot.kde(by='cat', y='value', subcoordinate_y={'subcoordinate_scale': 1.5}, legend=False, color="gray")

:NdOverlay   [cat]
   :Distribution   [value]   (Density)

More information about subcoordinate y-axis plots can be found in [HoloViews' customizing plots guide](https://holoviews.org/user_guide/Customizing_Plots.html#subcoordinate-y-axis) and in its [gallery](https://holoviews.org/gallery/demos/bokeh/multichannel_timeseries_viewer.html).

## New hover options: `hover_tooltips` and `hover_formatters`

The `hover_tooltips` and `hover_formatters` keywords have been added to complement `hover` and `hover_cols`. In order to customize the **Bokeh** hover tool, hvPlot users previously had to import and configure the `HoverTool` model from Bokeh's API. With these two new options added in HoloViews 1.19.0, you can now directly customize the hover tool wihout any additional import. Find out more about the values accepted by these options in [HoloViews' Plotting with Bokeh guide](https://holoviews.org/user_guide/Plotting_with_Bokeh.html#hover-tools).

In [12]:
import hvplot.pandas  # noqa
from bokeh.sampledata.periodic_table import elements

elements.head(2)

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,EA,standard state,bonding type,melting point,boiling point,density,metal,year discovered,group,period
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.2,37.0,NaN,120.0,...,-73.0,gas,diatomic,14.0,20.0,0.00009,nonmetal,1766,1,1
1,2,He,Helium,4.002602,#D9FFFF,1s2,NaN,32.0,NaN,140.0,...,0.0,gas,atomic,NaN,4.0,0.00000,noble gas,1868,18,1


In [13]:
elements.sort_values('metal').hvplot.points(
    'electronegativity', 'density', by='metal',
    hover_cols=['name', 'symbol', 'CPK'],
    hover_tooltips=[
        'name',
        ('Symbol', '@symbol'),
        ('CPK', '$color[hex, swatch]:CPK'),
        ('Density',  '@density{%.2e}'),
    ],
    hover_formatters={
        '@{density}': 'printf',
    }
)

:NdOverlay   [metal]
   :Points   [electronegativity,density]   (name,symbol,CPK)

## Optimized Pandas index support

HoloViews 1.19.0 came with optimizations around how Pandas DataFrame indexes are handled, effectively no longer internally calling `.reset_index()`, which was affecting memory usage and speed. Following HoloViews, hvPlot's code base was adapted accordingly, making sure that in most cases `.reset_index()` is not called. This had the benefit to improve the handling of wide datasets too. No pretty plot for this enhancement 😊 But it's a change that touched some deeper part of the two code bases so we wanted everyone to be aware of it and [report](https://github.com/holoviz/hvplot/issues) any issues.

## Fixing "No output in jupyter"

An unfortunately too common [issue](https://github.com/holoviz/hvplot/issues/1232) when using hvPlot in a Jupyter Notebook was that sometimes the plots would not just show up no matter how hard you tried, even with after following the classic procedure: notebook cleaning + notebook saving + browser tab hard refresh 😔 The `import hvplot.<integration>` import mechanism is a convenient way to allow users to have to avoid running the HoloViews/Panel extensions (e.g. `hv.extension('bokeh')`). However, since Python imports are cached, only the first import actually embeds the extension JavaScript code, meaning that if you re-run the cell(s) containing `import hvplot.pandas` (or some other integration) then the JavaScript will no longer be available and on subsequent reloads/re-runs of the notebook plots may not appear.

In this release, hvPlot adds an IPython hook which simply deletes the imported modules before *every* cell execution. This is a big hammer but the best we could find! Don't hesitate to provide us [feedback](https://github.com/holoviz/hvplot/issues) if you encounter any issue related to this change.

## Update of the minimum version of the dependencies

This regular maintenance practice had not been done in a while. Most notably, hvPlot now depends on `holoviews>=1.19.0` (compared to `>=0.11.0` previously) to ensure its users benefits from all the new features HoloViews has made available in the recent years. Additionally, hvPlot 0.11 requires Python 3.9 and above. For more details check the diff of the [Pull Request](https://github.com/holoviz/hvplot/pull/1355/files#diff-50c86b7ed8ac2cf95bd48334961bf0530cdc77b5a56f852c5c61b89d735fd711) that implemented this change.

---

Join us on [Github](https://github.com/holoviz/hvplot), [Discourse](https://discourse.holoviz.org/c/hvplot/8) or [Discord](https://discord.gg/AXRHnJU6sP) to help us improve hvPlot. Happy plotting 😊